Understand and compare the three main approaches to Transfer Learning – Feature Extraction, Partial Fine-Tuning, and Full Fine-Tuning – using a pretrained ResNet model on Fashion MNIST.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = models.resnet18(weights="IMAGENET1K_V1")

Strategy 1: Feature Extraction (Fixed Base)

In [ ]:
# Freeze all layers
model = base_model
for param in model.parameters():
    param.requires_grad = False

# Replace the final classification layer
model.fc = nn.Linear(512, 10)
model = model.to(device)

# Train only the new classifier head
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(2):  # few epochs for demonstration
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

Strategy 2: Partial Fine-Tuning

In [ ]:
# Unfreeze only the last residual block (layer4) and the final classifier
model = base_model
for name, param in model.named_parameters():
    param.requires_grad = ("layer4" in name) or ("fc" in name)

# Replace the classifier
model.fc = nn.Linear(512, 10)
model = model.to(device)

# Train the unfrozen layers with a smaller learning rate
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
criterion = nn.CrossEntropyLoss()

Strategy 3: Full Fine-Tuning

In [ ]:
# Unfreeze all layers
model = base_model
for param in model.parameters():
    param.requires_grad = True

# Replace the classifier again
model.fc = nn.Linear(512, 10)
model = model.to(device)

# Use a very small learning rate to avoid catastrophic forgetting
optimizer = optim.Adam(model.parameters(), lr=1e-6)
criterion = nn.CrossEntropyLoss()


Evaluate Models

In [ ]:
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

# Compute accuracies after each stage (assuming separate models)
acc_fixed   = evaluate(model)   # Feature extraction
acc_partial = evaluate(model)   # Partial fine-tuning
acc_full    = evaluate(model)   # Full fine-tuning

print(f"Feature Extraction Accuracy: {acc_fixed:.3f}")
print(f"Partial Fine-Tuning Accuracy: {acc_partial:.3f}")
print(f"Full Fine-Tuning Accuracy: {acc_full:.3f}")

Which strategy achieved the highest accuracy, and why?

In what situations would you prefer each approach (e.g., small dataset, limited compute, domain similarity)?

Could you design a hybrid approach – starting with feature extraction and gradually unfreezing layers?